In [3]:
import pandas as pd
import os

In [4]:
# This is where our downloaded images and metadata live locally
DATA_PATH = r'C:\\Users\\mfarj\\Documents\\ss_data\\'
dataset_path = os.path.join(DATA_PATH, 'train_species_only_singles.csv')
images_df = pd.read_csv(dataset_path)
print(f'Images df head: \n{images_df.head()}')
print(f'Number of images: {len(images_df)}')

Images df head: 
                          file_path_rel  encoded_species
0  S9/B03/B03_R1/S9_B03_R1_IMAG0250.JPG                1
1  S9/B03/B03_R1/S9_B03_R1_IMAG0251.JPG                1
2  S9/B03/B03_R1/S9_B03_R1_IMAG0252.JPG                1
3  S9/B03/B03_R1/S9_B03_R1_IMAG0256.JPG                1
4  S9/B03/B03_R1/S9_B03_R1_IMAG0258.JPG                1
Number of images: 193978


In [31]:
images_df['capture_id'] = images_df.file_path_rel.str.split('/').str[-1]
multiple_captures = images_df[images_df.duplicated(['capture_id'])]
multiple_captures.head()

,file_path_rel,encoded_species,capture_id


In [7]:
df_full_images = pd.read_csv(DATA_PATH + '\SnapshotSerengeti_v2_1_images.csv')
df_full_annotations = pd.read_csv(DATA_PATH + '\SnapshotSerengeti_v2_1_annotations.csv')
df_full_images.rename(columns = {'Unnamed: 0':'seq_id'}, inplace=True)
df_full_images.index = df_full_images.capture_id
df_full_annotations.rename(columns = {'Unnamed: 0':'seq_id'}, inplace=True)

print('Annotated Sequences: ' + str(df_full_annotations.capture_id.nunique()))
# number of sequences
print('Unique Sequences: ' + str(df_full_images.capture_id.nunique()))

#reduce to first frame only for all sequences. Not doing this now and taking all the images in
#df_full_images = df_full_images.sort_values('file_name').groupby('seq_id').first()

df_full_images['season'] = df_full_images.capture_id.map(lambda x: x.split('#')[0])

print(f'{df_full_images.season.value_counts().sort_index()}')

df_full_annotations.groupby(['question__species'])['seq_id'].count()


df_full_annotations.groupby(['question__species'])['seq_id'].count()

df_full_images = df_full_images[df_full_images.season.isin(['SER_S9'])]
df_full_annotations = df_full_annotations[df_full_annotations.capture_id.isin(df_full_images.capture_id)]
#
# ss_data_host = str('https://lilablobssc.blob.core.windows.net/snapshotserengeti-unzipped')
# ss_data_authentication = str('?st=2020-01-01T00%3A00%3A00Z&se=2034-01-01T00%3A00%3A00Z&sp=rl&sv=2019-07-07&sr=c&sig=/DGPd%2B9WGFt6HgkemDFpo2n0M1htEXvTq9WoHlaH7L4%3D')
#
# df_full_images['file_name'] = df_full_images.apply(
#     lambda x: (ss_data_host + str(f'/{x.image_path_rel}') + ss_data_authentication), axis=1
# )


C:\Users\mfarj\.conda\envs\thesis\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (8,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Annotated Sequences: 2659222
Unique Sequences: 2659222
SER_S1     411414
SER_S10    685481
SER_S11    499401
SER_S2     573200
SER_S3     392507
SER_S4     531554
SER_S5     827224
SER_S6     462846
SER_S7     832153
SER_S8     980256
SER_S9     982404
Name: season, dtype: int64


In [11]:
df_full_images['image_name'] = df_full_images.image_path_rel.str.split('/').str[-1]
df_full_images.head()

,seq_id,capture_id,image_rank_in_capture,image_path_rel,season,image_name
capture_id,,,,,,
SER_S9#B03#1#1,0,SER_S9#B03#1#1,1,S9/B03/B03_R1/S9_B03_R1_IMAG0001.JPG,SER_S9,S9_B03_R1_IMAG0001.JPG
SER_S9#B03#1#1,1,SER_S9#B03#1#1,2,S9/B03/B03_R1/S9_B03_R1_IMAG0002.JPG,SER_S9,S9_B03_R1_IMAG0002.JPG
SER_S9#B03#1#2,2,SER_S9#B03#1#2,1,S9/B03/B03_R1/S9_B03_R1_IMAG0003.JPG,SER_S9,S9_B03_R1_IMAG0003.JPG
SER_S9#B03#1#3,3,SER_S9#B03#1#3,1,S9/B03/B03_R1/S9_B03_R1_IMAG0004.JPG,SER_S9,S9_B03_R1_IMAG0004.JPG
SER_S9#B03#1#3,4,SER_S9#B03#1#3,2,S9/B03/B03_R1/S9_B03_R1_IMAG0005.JPG,SER_S9,S9_B03_R1_IMAG0005.JPG


In [17]:
df_full = df_full_images.join(df_full_annotations.set_index('capture_id'), how='inner', on=None, lsuffix='_x', rsuffix='_y')

print(f'Columns: {df_full.columns}')
print(f"Question species: {df_full['question__species']}")

Columns: Index(['seq_id_x', 'capture_id', 'image_rank_in_capture', 'image_path_rel',
       'season_x', 'image_name', 'seq_id_y', 'season_y', 'site', 'roll',
       'capture', 'capture_date_local', 'capture_time_local', 'subject_id',
       'question__species', 'question__count_max', 'question__count_median',
       'question__count_min', 'question__standing', 'question__resting',
       'question__moving', 'question__eating', 'question__interacting',
       'question__young_present', 'p_users_identified_this_species',
       'pielous_evenness_index', 'question__horns_visible'],
      dtype='object')
Question species: capture_id
SER_S9#B03#1#1       blank
SER_S9#B03#1#1       blank
SER_S9#B03#1#10      blank
SER_S9#B03#1#10      blank
SER_S9#B03#1#10      blank
                    ...   
SER_S9#U11#1#6       human
SER_S9#U11#1#6       human
SER_S9#U11#1#7     cheetah
SER_S9#U11#1#8       blank
SER_S9#U11#1#9       blank
Name: question__species, Length: 991858, dtype: object


In [19]:
df_full = df_full.loc[~df_full.question__species.str.contains('blank')]
len(df_full)

217848

In [24]:
df_missing_left = pd.merge(df_full, images_df, how='left', left_on= "image_name", right_on="capture_id")
print(df_missing_left.columns)

Index(['seq_id_x', 'capture_id_x', 'image_rank_in_capture', 'image_path_rel',
       'season_x', 'image_name', 'seq_id_y', 'season_y', 'site', 'roll',
       'capture', 'capture_date_local', 'capture_time_local', 'subject_id',
       'question__species', 'question__count_max', 'question__count_median',
       'question__count_min', 'question__standing', 'question__resting',
       'question__moving', 'question__eating', 'question__interacting',
       'question__young_present', 'p_users_identified_this_species',
       'pielous_evenness_index', 'question__horns_visible', 'file_path_rel',
       'encoded_species', 'capture_id_y'],
      dtype='object')


In [26]:
df_missing_left = df_missing_left[['seq_id_x', 'capture_id_x', 'image_rank_in_capture', 'season_x', 'subject_id', 'question__species', 'file_path_rel', 'encoded_species', 'capture_id_y']]
print(f'DF MISSING: {df_missing_left.head()}')
print(f'DF MISSING columns: {df_missing_left.columns}')

DF MISSING:    seq_id_x      capture_id_x  image_rank_in_capture season_x  subject_id  \
0       249  SER_S9#B03#1#101                      1   SER_S9  ASG001j6xd   
1       250  SER_S9#B03#1#101                      2   SER_S9  ASG001j6xd   
2       251  SER_S9#B03#1#101                      3   SER_S9  ASG001j6xd   
3       255  SER_S9#B03#1#103                      1   SER_S9  ASG001j6xe   
4       256  SER_S9#B03#1#103                      2   SER_S9  ASG001j6xe   

  question__species                         file_path_rel  encoded_species  \
0        wildebeest  S9/B03/B03_R1/S9_B03_R1_IMAG0250.JPG              1.0   
1        wildebeest  S9/B03/B03_R1/S9_B03_R1_IMAG0251.JPG              1.0   
2        wildebeest  S9/B03/B03_R1/S9_B03_R1_IMAG0252.JPG              1.0   
3        wildebeest  S9/B03/B03_R1/S9_B03_R1_IMAG0256.JPG              1.0   
4        wildebeest                                   NaN              NaN   

             capture_id_y  
0  S9_B03_R1_IMAG0250.JPG  


In [29]:
df_missing= df_missing_left[df_missing_left.isnull().any(axis=1)]
print(f'DF MISSING: {df_missing.head()}')
print(f'DF MISSING columns: {df_missing.columns}')

DF MISSING:      seq_id_x      capture_id_x  image_rank_in_capture season_x  subject_id  \
4         256  SER_S9#B03#1#103                      2   SER_S9  ASG001j6xe   
279         2    SER_S9#B03#1#2                      1   SER_S9  ASG001lfjx   
347        56   SER_S9#B03#1#24                      1   SER_S9  ASG001i812   
348        56   SER_S9#B03#1#24                      1   SER_S9  ASG001i812   
349        57   SER_S9#B03#1#24                      2   SER_S9  ASG001i812   

    question__species file_path_rel  encoded_species capture_id_y  
4          wildebeest           NaN              NaN          NaN  
279             human           NaN              NaN          NaN  
347        wildebeest           NaN              NaN          NaN  
348             zebra           NaN              NaN          NaN  
349        wildebeest           NaN              NaN          NaN  
DF MISSING columns: Index(['seq_id_x', 'capture_id_x', 'image_rank_in_capture', 'season_x',
       'subje